# 리뷰 데이터
- 리뷰_교보베스트_최종.csv
- 리뷰_교보비베스트_최종.csv
- 리뷰_알라딘베스트_최종.csv
- 리뷰_알라딘비베스트_최종.csv
- 리뷰_영풍베스트_최종.csv
- 리뷰_영풍비베스트_최종.csv
- 리뷰_예사베스트_정말최종.csv
- 리뷰_예사비베스트_정말최종.csv

# 리뷰 데이터_감성분석용
- 리뷰_교보베스트_감성분석용.csv
- 리뷰_교보비베스트_감성분석용.csv
- 리뷰_알라딘베스트_감성분석용.csv
- 리뷰_알라딘비베스트_감성분석용.csv
- 리뷰_영풍베스트_감성분석용.csv
- 리뷰_영풍비베스트_감성분석용.csv
- 리뷰_예사베스트_감성분석용.csv
- 리뷰_예사비베스트_감성분석용.csv

# 모델학습

In [4]:
import numpy as np
import pandas as pd

도서리뷰샘플 = pd.read_csv('리뷰_수작업_29718_도서.csv')
도서리뷰샘플

,review,label
0,가격에 비해 이야기가 흥미가 떨어져요........................,0
1,그냥 술술 읽히는데 남은건..? 뭐지,0
2,책읽어보고 좋아서 할미언니강연도 갔어요\n넘좋앗어용,1
3,"마음을 공부하고 마음을 다스린다는 것\n다산 정약용 선생, 조선 최고의 실학자이자 ...",1
4,연애라는 주제에 관심이 이전부터 많았던 터라 그동안 갖고있던 생각이랑 알고있던사실들...,0
...,...,...
29713,아이에게도 도움이 될것 같아요.,1
29714,도서관에서빌려읽으세요,0
29715,아이가 재미있어하며 잘 봐요,1
29716,필사해도 좋을 것 같습니다.,1


In [7]:
train_df = pd.read_csv('ratings_train.txt', sep='\t') #sep => 원본텍스트 파일은 쉼표가 아닌 탭('\t')으로 칼럼이 분리되어 있다.
test_df = pd.read_csv('ratings_test.txt', sep='\t') #sep => 원본텍스트 파일은 쉼표가 아닌 탭('\t')으로 칼럼이 분리되어 있다.

In [8]:
train_df = train_df.dropna(subset=['document'])
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df = train_df[~train_df['document'].str.contains(r'^\d+$')]
test_df = test_df.dropna(subset=['document'])
test_df.drop_duplicates(subset=['document'], inplace=True) # 추가
test_df = test_df[~test_df['document'].str.contains(r'^\d+$')]

In [9]:
### 제작된 도서리뷰 데이터 추가

# train 과 test df의 id열 삭제
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

# 도서리뷰샘플 열이름 변경
도서리뷰샘플.rename(columns={'review': 'document'}, inplace=True)

# 도서리뷰샘플을 3:1비율 무작위 분할
from sklearn.model_selection import train_test_split
도서리뷰샘플_train, 도서리뷰샘플_test = train_test_split(도서리뷰샘플, test_size=0.2, random_state=42)

# train_df 에 도서리뷰샘플_train 이어붙이기
train_df = pd.concat([train_df, 도서리뷰샘플_train], ignore_index=True)

# test_df 에 도서리뷰샘플_test 이어붙이기
test_df = pd.concat([test_df, 도서리뷰샘플_test], ignore_index=True)

In [10]:
# 불용어 처리 + 형태소 분석

from konlpy.tag import Okt
stopwords = set([
    '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '를', '으로', '자', '에', '와', '한',
    '하다', '에', '는', '가', '이다', '을', '를', '이', '다', '그', '가', '에', '한', '하다', '것',
    '등', '그리고', '나', '아니', '있다', '없다', '이다', '같다', '때문에', '등', '그', '고', '의'
])

okt = Okt()
def okt_token(text):
    tokens_ko = okt.morphs(text, stem=True)
    # 불용어 처리
    tokens_ko = [word for word in tokens_ko if word not in stopwords]
    return tokens_ko

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=okt_token, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
tfidf_matrix_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3199965 stored elements and shape (169931, 128821)>

In [13]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행.
lg_clf = LogisticRegression(random_state=0, solver='liblinear')

# Parameter C 최적화를 위해 GridSearchCV 를 이용.
# C는 규제(Regularization): 규제는 모델의 복잡성을 줄여 과적합(overfitting)을 방지하는 방법입니다.
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )

#어차피 학습과 테스트 데이터가 나누어져 있기 때문에 따로 스플릿 할 필요는 없습니다.
grid_cv.fit(tfidf_matrix_train , train_df['label'] ) #학습
print(grid_cv.best_params_ , round(grid_cv.best_score_,4)) # 최적의 파라미터 도출

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8412


In [14]:
from sklearn.metrics import accuracy_score

# 학습된 TfidfVectorizer를 이용하여 test_df를 TF-IDF 값으로 Feature 변환함.
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_esimator = grid_cv.best_estimator_
preds = best_esimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.8530266167979955


In [15]:
text_sample = '쫌 재미있네요'
# 자연어 그 자체로는 예측할 수 없다.
# 학습했을때와 동일하게 문장을 형태소 분석/벡터화.
text_tfidf = tfidf_vect.transform([text_sample])
prediction = best_esimator.predict(text_tfidf)
prediction_prob = best_esimator.predict_proba(text_tfidf)
prediction_prob

array([[0.08070856, 0.91929144]])

In [ ]:
def senti_pred(text_sample):
    text_tfidf = tfidf_vect.transform([text_sample])
    prediction = best_esimator.predict(text_tfidf)
    prediction_prob = best_esimator.predict_proba(text_tfidf)
    if prediction[0] == 1:
        senti = '긍정'
    else:
        senti = '부정'
    score = prediction_prob[0][prediction[0]]
    print(f'{text_sample}에 대한 예측: {senti}, 확률(%): {round(score * 100, 2)}')

In [17]:
import pandas as pd
df = pd.read_csv('리뷰_예사베스트_감성분석용.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268466 entries, 0 to 268465
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  268405 non-null  object
dtypes: object(1)
memory usage: 2.0+ MB


In [34]:
df = df.dropna(subset=['review'])

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 268405 entries, 0 to 268465
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  268405 non-null  object
dtypes: object(1)
memory usage: 4.1+ MB


In [39]:
df1 = pd.read_csv('리뷰_예사비베스트_감성분석용.csv')

In [42]:
df1 = df1.dropna(subset=['review'])

In [37]:
def senti_pred(text_sample):
    text_tfidf = tfidf_vect.transform([text_sample])
    prediction = best_esimator.predict(text_tfidf)
    prediction_prob = best_esimator.predict_proba(text_tfidf)

    if prediction[0] == 1:
        senti = '긍정'
    else:
        senti = '부정'

    score = round(prediction_prob[0][prediction[0]] * 100, 2)
    return senti, score

In [43]:
# 'review' 열에 함수 적용 → 결과를 2개의 컬럼으로 분리해서 저장
df[['sentiment', 'sentiment_score']] = df['review'].apply(
    lambda x: pd.Series(senti_pred(x))
)

C:\Users\user\AppData\Local\Temp\ipykernel_12460\3962587223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['sentiment', 'sentiment_score']] = df['review'].apply(
C:\Users\user\AppData\Local\Temp\ipykernel_12460\3962587223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['sentiment', 'sentiment_score']] = df['review'].apply(


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 268405 entries, 0 to 268465
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   review           268405 non-null  object 
 1   sentiment        268405 non-null  object 
 2   sentiment_score  268405 non-null  float64
dtypes: float64(1), object(2)
memory usage: 8.2+ MB


In [48]:
import pandas as pd

# 파일명 목록
file_list = [
    '리뷰_교보베스트_감성분석용.csv',
    '리뷰_교보비베스트_감성분석용.csv',
    '리뷰_알라딘베스트_감성분석용.csv',
    '리뷰_알라딘비베스트_감성분석용.csv',
    '리뷰_영풍베스트_감성분석용.csv',
    '리뷰_영풍비베스트_감성분석용.csv',
    '리뷰_예사베스트_감성분석용.csv',
    '리뷰_예사비베스트_감성분석용.csv'
]

# 처리된 결과를 저장할 딕셔너리
processed_dfs = {}

for file_name in file_list:
    print(f'📘 처리 중: {file_name}')

    # 1. 파일 불러오기
    df = pd.read_csv(file_name)

    # 2. 결측값 처리
    df = df.dropna(subset=['review'])

    # 3. 감성 분석 결과 추가
    df[['sentiment', 'sentiment_score']] = df['review'].apply(lambda x: pd.Series(senti_pred(x)))

    # 4. 결과 저장 (선택적으로 파일 저장도 가능)
    processed_dfs[file_name] = df
    df.to_csv(f'감성분석완료_{file_name}', index=False)  # 저장하고 싶을 경우 사용

    print(f'✅ 완료: {file_name} ({len(df)}건)')



📘 처리 중: 리뷰_교보베스트_감성분석용.csv
✅ 완료: 리뷰_교보베스트_감성분석용.csv (496069건)
📘 처리 중: 리뷰_교보비베스트_감성분석용.csv
✅ 완료: 리뷰_교보비베스트_감성분석용.csv (33118건)
📘 처리 중: 리뷰_알라딘베스트_감성분석용.csv
✅ 완료: 리뷰_알라딘베스트_감성분석용.csv (189485건)
📘 처리 중: 리뷰_알라딘비베스트_감성분석용.csv
✅ 완료: 리뷰_알라딘비베스트_감성분석용.csv (48707건)
📘 처리 중: 리뷰_영풍베스트_감성분석용.csv
✅ 완료: 리뷰_영풍베스트_감성분석용.csv (3034건)
📘 처리 중: 리뷰_영풍비베스트_감성분석용.csv
✅ 완료: 리뷰_영풍비베스트_감성분석용.csv (386건)
📘 처리 중: 리뷰_예사베스트_감성분석용.csv
✅ 완료: 리뷰_예사베스트_감성분석용.csv (268405건)
📘 처리 중: 리뷰_예사비베스트_감성분석용.csv
✅ 완료: 리뷰_예사비베스트_감성분석용.csv (73223건)


In [84]:
import pandas as pd
df = pd.read_csv('감성분석완료_리뷰_영풍비베스트_감성분석용.csv')
test = pd.read_csv('[영풍] 비베스트셀러 리뷰_1차정제.csv')

In [85]:
df

,review,sentiment,sentiment_score
0,1권만 사서 읽어봤는데요 별로 2권까지 사서 읽어보고 싶지 않네요 좀 실망입니다,부정,93.59
1,백석의 소설과 수필의 정수를 맛볼 진수성찬이 차려졌다 백석 시인의 강한 개성을 보여...,긍정,98.03
2,백석 고유의 어휘를 살리면서도 곳곳에 작품을 잘 이해할 수 있도록 돕는 어휘 설명과...,긍정,85.15
3,백석은 시와 마찬가지로 소설과 수필에서도 허투루 쓴 작품이 없다 소설은 물론 수필에...,긍정,76.09
4,시나 소설을 비롯한 문학작품을 읽게 되면 나 혼자만이 오롯이 느끼는 감정도 소중하지...,긍정,96.09
...,...,...,...
381,이 전에 공간은 음식점이면 음식을 먹기만을 위한 공간으로만 정의가 되었고 그곳을 가...,긍정,97.18
382,자주가는 연남동에 참기름 파는 공간을 새로운 시각으로 설명해주어서 색다르고 신선했어...,긍정,94.47
383,공간에 대한 의미를 알 수 있게 되어서 좋아요 책에 나온 곳들 한번씩 다 방문해보고싶네용,긍정,80.11
384,책이 참 쉽게 구성되어 있어서 좋습니다 그리고 기존의 책들은 기계도면이나 건축도면 ...,긍정,85.30


In [86]:
test = test.dropna(subset=['review'])

In [87]:
test

,isbn,review,rating
0,9788957591840,1권만 사서 읽어봤는데요 별로 2권까지 사서 읽어보고 싶지 않네요 좀 실망입니다,2
1,9788954657983,백석의 소설과 수필의 정수를 맛볼 진수성찬이 차려졌다 백석 시인의 강한 개성을 보여...,5
2,9788954657983,백석 고유의 어휘를 살리면서도 곳곳에 작품을 잘 이해할 수 있도록 돕는 어휘 설명과...,5
3,9788954657983,백석은 시와 마찬가지로 소설과 수필에서도 허투루 쓴 작품이 없다 소설은 물론 수필에...,5
4,9788954657983,시나 소설을 비롯한 문학작품을 읽게 되면 나 혼자만이 오롯이 느끼는 감정도 소중하지...,5
...,...,...,...
382,9791191157017,이 전에 공간은 음식점이면 음식을 먹기만을 위한 공간으로만 정의가 되었고 그곳을 가...,5
383,9791191157017,자주가는 연남동에 참기름 파는 공간을 새로운 시각으로 설명해주어서 색다르고 신선했어...,5
384,9791191157017,공간에 대한 의미를 알 수 있게 되어서 좋아요 책에 나온 곳들 한번씩 다 방문해보고싶네용,5
385,9788931556179,책이 참 쉽게 구성되어 있어서 좋습니다 그리고 기존의 책들은 기계도면이나 건축도면 ...,5


In [88]:
df1 = pd.merge(df, test[['review', 'isbn', 'rating']], on='review', how='left')

In [89]:
df1 = df1[['isbn', 'review', 'rating', 'sentiment', 'sentiment_score']]

In [90]:
df1

,isbn,review,rating,sentiment,sentiment_score
0,9788957591840,1권만 사서 읽어봤는데요 별로 2권까지 사서 읽어보고 싶지 않네요 좀 실망입니다,2,부정,93.59
1,9788954657983,백석의 소설과 수필의 정수를 맛볼 진수성찬이 차려졌다 백석 시인의 강한 개성을 보여...,5,긍정,98.03
2,9788954657983,백석 고유의 어휘를 살리면서도 곳곳에 작품을 잘 이해할 수 있도록 돕는 어휘 설명과...,5,긍정,85.15
3,9788954657983,백석은 시와 마찬가지로 소설과 수필에서도 허투루 쓴 작품이 없다 소설은 물론 수필에...,5,긍정,76.09
4,9788954657983,시나 소설을 비롯한 문학작품을 읽게 되면 나 혼자만이 오롯이 느끼는 감정도 소중하지...,5,긍정,96.09
...,...,...,...,...,...
381,9791191157017,이 전에 공간은 음식점이면 음식을 먹기만을 위한 공간으로만 정의가 되었고 그곳을 가...,5,긍정,97.18
382,9791191157017,자주가는 연남동에 참기름 파는 공간을 새로운 시각으로 설명해주어서 색다르고 신선했어...,5,긍정,94.47
383,9791191157017,공간에 대한 의미를 알 수 있게 되어서 좋아요 책에 나온 곳들 한번씩 다 방문해보고싶네용,5,긍정,80.11
384,9788931556179,책이 참 쉽게 구성되어 있어서 좋습니다 그리고 기존의 책들은 기계도면이나 건축도면 ...,5,긍정,85.30


In [91]:
df1.drop_duplicates(subset=['review'], inplace=True)

In [92]:
df1

,isbn,review,rating,sentiment,sentiment_score
0,9788957591840,1권만 사서 읽어봤는데요 별로 2권까지 사서 읽어보고 싶지 않네요 좀 실망입니다,2,부정,93.59
1,9788954657983,백석의 소설과 수필의 정수를 맛볼 진수성찬이 차려졌다 백석 시인의 강한 개성을 보여...,5,긍정,98.03
2,9788954657983,백석 고유의 어휘를 살리면서도 곳곳에 작품을 잘 이해할 수 있도록 돕는 어휘 설명과...,5,긍정,85.15
3,9788954657983,백석은 시와 마찬가지로 소설과 수필에서도 허투루 쓴 작품이 없다 소설은 물론 수필에...,5,긍정,76.09
4,9788954657983,시나 소설을 비롯한 문학작품을 읽게 되면 나 혼자만이 오롯이 느끼는 감정도 소중하지...,5,긍정,96.09
...,...,...,...,...,...
381,9791191157017,이 전에 공간은 음식점이면 음식을 먹기만을 위한 공간으로만 정의가 되었고 그곳을 가...,5,긍정,97.18
382,9791191157017,자주가는 연남동에 참기름 파는 공간을 새로운 시각으로 설명해주어서 색다르고 신선했어...,5,긍정,94.47
383,9791191157017,공간에 대한 의미를 알 수 있게 되어서 좋아요 책에 나온 곳들 한번씩 다 방문해보고싶네용,5,긍정,80.11
384,9788931556179,책이 참 쉽게 구성되어 있어서 좋습니다 그리고 기존의 책들은 기계도면이나 건축도면 ...,5,긍정,85.30


In [93]:
df1.to_csv('리뷰최종_영풍비베스트.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('리뷰최종_교보베스트.csv')

In [96]:
df.groupby('isbn')['rating'].mean()

isbn
9788901098272    4.876923
9788901116105    5.000000
9788901165851    4.870370
9788901216270    4.744186
9788901222141    4.771429
                   ...   
9791198960009    4.333333
9791198984814    4.857143
9791198987600    4.914729
9791198999108    4.384615
9791199015807    4.937759
Name: rating, Length: 9184, dtype: float64

# 리뷰데이터
- 누락, 중복 행 삭제
- 특수기호, 이모티콘 등 삭제
- 이상한 데이터 행 삭제
- 감성분석 완료

# 리뷰 데이터 파일
- [교보] 베스트셀러 리뷰_1차정제.csv
- [교보] 비베스트셀러 리뷰_1차정제.csv
- [알라딘] 베스트셀러 리뷰_1차정제.csv
- [알라딘] 비베스트셀러 리뷰_1차정제.csv
- [영풍] 베스트셀러 리뷰_1차정제.csv
- [영풍] 비베스트셀러 리뷰_1차정제.csv
- 리뷰_예사베스트_정말최종.csv
- 리뷰_예사비베스트_정말최종.csv

# 최종 리뷰 데이터
- 리뷰_예사베스트

# 감성점수 산출
도서평가점수 = 리뷰수*0.2 + 감성점수*0.5 + 별점*0.3
- 감성점수는 각 플랫폼별로 ISBN별 감성점수의 평균
- 별점 또한 각 플랫폼별로 ISBN별 별점의 평균
- 산출된 도서평가점수는 도서메타데이터의 포함
ex) kyobo_score
